## 程序框架


**程序目录**

- include/
- src/
- main.cc
- CMakeLists.txt
- vis.mac
- init_vis.mac

*CMakeLists.txt* 原则上不需要改动，如果需要将主目录下的文件复制到编译的文件夹，则参考以下代码进行修改。

```cmake
# 以下代码将 init.mac、init_vis.mac、vis.mac、wu.in、wu.out 这5个文件复制到
set(COPY_SCRIPTS
   init.mac
   init_vis.mac
   vis.mac
   wu.in
   wu.out
  )

foreach(_script ${COPY_SCRIPTS})
  configure_file(
    ${PROJECT_SOURCE_DIR}/${_script}
    ${PROJECT_BINARY_DIR}/${_script}
    COPYONLY
    )
endforeach()
```


**main.cc**

```cpp
int main(int argc,char** argv)
{
  // 随机数设置 如果需要的话
  // G4Random::setTheEngine(new CLHEP::RanecuEngine);
  // G4int seconds =  time(NULL);
  // G4Random::setTheSeed(seconds);


  // Construct the default run manager
  G4MTRunManager* mtrunManager = NULL;

  mtrunManager = new G4MTRunManager;
  mtrunManager->SetNumberOfThreads(2);
  // mtrunManager->SetUserInitialization(new wuWorkerInitialization);


  // Set mandatory initialization classes ，410版本的框架是这样的，ActionInitialization来管理。
  //
  // Detector construction
  mtrunManager->SetUserInitialization(new wuDetectorConstruction());


  // Physics list
  // G4VModularPhysicsList* physicsList = new wuPhysicsList();
  mtrunManager->SetUserInitialization(new FTFP_BERT_HP());
  
  // User action initialization
  mtrunManager->SetUserInitialization(new wuActionInitialization());


  // Initialize G4 kernel
  mtrunManager->Initialize();


    
  G4VisManager* visManager = 0;
  G4UImanager* UImanager = G4UImanager::GetUIpointer();

  G4String commandopt = argv[1];
  if(commandopt == "vis.mac")// 开启图形界面模式
    {
      visManager = new G4VisExecutive;
      visManager->Initialize();

      G4UIExecutive* ui = new G4UIExecutive(argc, argv);
      UImanager->ApplyCommand("/control/execute vis.mac");
      ui->SessionStart();
      delete ui;
    }
  else
    {
      if(commandopt == "-l")// 开启命令行模式
	{  
	  G4UIsession* session = new G4UIterminal(new G4UItcsh);
	  session->SessionStart();
	  delete session;
	}
      else
	{// 无图形界面执行脚本模式
	  G4String command = "/control/execute ";
	  G4String fileName = argv[1];
	  UImanager->ApplyCommand(command + fileName);
	}
    }

  
  // Job termination
  // Free the store: user actions, physics_list and detector_description are
  // owned and deleted by the run manager, so they should not be deleted 
  // in the main() program !
  
  if(visManager != 0)
    delete visManager;

  delete mtrunManager;

  return 0;
}
```




In [1]:
!jupyter nbconvert MAIN.ipynb --to html

[NbConvertApp] Converting notebook MAIN.ipynb to html

[NbConvertApp] Writing 557312 bytes to MAIN.html

